In [ ]:
#!/bin/env python

import argparse
from itertools import product
import math
import nltk
from pathlib import Path

from preprocess import preprocess


def load_data(data_dir):
    """Load train and test corpora from a directory.

    Directory must contain two files: train.txt and test.txt.
    Newlines will be stripped out. 

    Args:
        data_dir (Path) -- pathlib.Path of the directory to use. 

    Returns:
        The train and test sets, as lists of sentences.

    """
    train_path = data_dir.joinpath('train.txt').absolute().as_posix()
    test_path  = data_dir.joinpath('test.txt').absolute().as_posix()

    with open(train_path, 'r') as f:
        train = [l.strip() for l in f.readlines()]
    with open(test_path, 'r') as f:
        test = [l.strip() for l in f.readlines()]
    return train, test


class LanguageModel(object):
    """An n-gram language model trained on a given corpus.
    
    For a given n and given training corpus, constructs an n-gram language
    model for the corpus by:
    1. preprocessing the corpus (adding SOS/EOS/UNK tokens)
    2. calculating (smoothed) probabilities for each n-gram

    Also contains methods for calculating the perplexity of the model
    against another corpus, and for generating sentences.

    Args:
        train_data (list of str): list of sentences comprising the training corpus.
        n (int): the order of language model to build (i.e. 1 for unigram, 2 for bigram, etc.).
        laplace (int): lambda multiplier to use for laplace smoothing (default 1 for add-1 smoothing).

    """

    def __init__(self, train_data, n, laplace=1):
        self.n = n
        self.laplace = laplace
        self.tokens = preprocess(train_data, n)
        self.vocab  = nltk.FreqDist(self.tokens)
        self.model  = self._create_model()
        self.masks  = list(reversed(list(product((0,1), repeat=n))))

    def _smooth(self):
        """Apply Laplace smoothing to n-gram frequency distribution.
        
        Here, n_grams refers to the n-grams of the tokens in the training corpus,
        while m_grams refers to the first (n-1) tokens of each n-gram.

        Returns:
            dict: Mapping of each n-gram (tuple of str) to its Laplace-smoothed 
            probability (float).

        """
        vocab_size = len(self.vocab)

        n_grams = nltk.ngrams(self.tokens, self.n)
        n_vocab = nltk.FreqDist(n_grams)

        m_grams = nltk.ngrams(self.tokens, self.n-1)
        m_vocab = nltk.FreqDist(m_grams)

        def smoothed_count(n_gram, n_count):
            m_gram = n_gram[:-1]
            m_count = m_vocab[m_gram]
            return (n_count + self.laplace) / (m_count + self.laplace * vocab_size)

        return { n_gram: smoothed_count(n_gram, count) for n_gram, count in n_vocab.items() }

    def _create_model(self):
        """Create a probability distribution for the vocabulary of the training corpus.
        
        If building a unigram model, the probabilities are simple relative frequencies
        of each token with the entire corpus.

        Otherwise, the probabilities are Laplace-smoothed relative frequencies.

        Returns:
            A dict mapping each n-gram (tuple of str) to its probability (float).

        """
        if self.n == 1:
            num_tokens = len(self.tokens)
            return { (unigram,): count / num_tokens for unigram, count in self.vocab.items() }
        else:
            return self._smooth()

    def _convert_oov(self, ngram):
        """Convert, if necessary, a given n-gram to one which is known by the model.

        Starting with the unmodified ngram, check each possible permutation of the n-gram
        with each index of the n-gram containing either the original token or <UNK>. Stop
        when the model contains an entry for that permutation.

        This is achieved by creating a 'bitmask' for the n-gram tuple, and swapping out
        each flagged token for <UNK>. Thus, in the worst case, this function checks 2^n
        possible n-grams before returning.

        Returns:
            The n-gram with <UNK> tokens in certain positions such that the model
            contains an entry for it.

        """
        mask = lambda ngram, bitmask: tuple((token if flag == 1 else "<UNK>" for token,flag in zip(ngram, bitmask)))

        ngram = (ngram,) if type(ngram) is str else ngram
        for possible_known in [mask(ngram, bitmask) for bitmask in self.masks]:
            if possible_known in self.model:
                return possible_known

    def perplexity(self, test_data):
        """Calculate the perplexity of the model against a given test corpus.
        
        Args:
            test_data (list of str): sentences comprising the training corpus.
        Returns:
            The perplexity of the model as a float.
        
        """
        test_tokens = preprocess(test_data, self.n)
        test_ngrams = nltk.ngrams(test_tokens, self.n)
        N = len(test_tokens)

        known_ngrams  = (self._convert_oov(ngram) for ngram in test_ngrams)
        probabilities = [self.model[ngram] for ngram in known_ngrams]

        return math.exp((-1/N) * sum(map(math.log, probabilities)))

    def _best_candidate(self, prev, i, without=[]):
        """Choose the most likely next token given the previous (n-1) tokens.

        If selecting the first word of the sentence (after the SOS tokens),
        the i'th best candidate will be selected, to create variety.
        If no candidates are found, the EOS token is returned with probability 1.

        Args:
            prev (tuple of str): the previous n-1 tokens of the sentence.
            i (int): which candidate to select if not the most probable one.
            without (list of str): tokens to exclude from the candidates list.
        Returns:
            A tuple with the next most probable token and its corresponding probability.

        """
        blacklist  = ["<UNK>"] + without
        candidates = ((ngram[-1],prob) for ngram,prob in self.model.items() if ngram[:-1]==prev)
        candidates = filter(lambda candidate: candidate[0] not in blacklist, candidates)
        candidates = sorted(candidates, key=lambda candidate: candidate[1], reverse=True)
        if len(candidates) == 0:
            return ("</s>", 1)
        else:
            return candidates[0 if prev != () and prev[-1] != "<s>" else i]
     
    def generate_sentences(self, num, min_len=12, max_len=24):
        """Generate num random sentences using the language model.

        Sentences always begin with the SOS token and end with the EOS token.
        While unigram model sentences will only exclude the UNK token, n>1 models
        will also exclude all other words already in the sentence.

        Args:
            num (int): the number of sentences to generate.
            min_len (int): minimum allowed sentence length.
            max_len (int): maximum allowed sentence length.
        Yields:
            A tuple with the generated sentence and the combined probability
            (in log-space) of all of its n-grams.

        """
        for i in range(num):
            sent, prob = ["<s>"] * max(1, self.n-1), 1
            while sent[-1] != "</s>":
                prev = () if self.n == 1 else tuple(sent[-(self.n-1):])
                blacklist = sent + (["</s>"] if len(sent) < min_len else [])
                next_token, next_prob = self._best_candidate(prev, i, without=blacklist)
                sent.append(next_token)
                prob *= next_prob
                
                if len(sent) >= max_len:
                    sent.append("</s>")

            yield ' '.join(sent), -1/math.log(prob)
       

if __name__ == '__main__':
    parser = argparse.ArgumentParser("N-gram Language Model")
    parser.add_argument('--data', type=str, required=True,
            help='Location of the data directory containing train.txt and test.txt')
    parser.add_argument('--n', type=int, required=True,
            help='Order of N-gram model to create (i.e. 1 for unigram, 2 for bigram, etc.)')
    parser.add_argument('--laplace', type=float, default=0.01,
            help='Lambda parameter for Laplace smoothing (default is 0.01 -- use 1 for add-1 smoothing)')
    parser.add_argument('--num', type=int, default=10,
            help='Number of sentences to generate (default 10)')
    args = parser.parse_args()

    # Load and prepare train/test data
    data_path = Path(args.data)
    train, test = load_data(data_path)

    print("Loading {}-gram model...".format(args.n))
    lm = LanguageModel(train, args.n, laplace=args.laplace)
    print("Vocabulary size: {}".format(len(lm.vocab)))

    print("Generating sentences...")
    for sentence, prob in lm.generate_sentences(args.num):
        print("{} ({:.5f})".format(sentence, prob))

    perplexity = lm.perplexity(test)
    print("Model perplexity: {:.3f}".format(perplexity))
    print("")


In [2]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
#import consine_similarity
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Define the directory paths
directories = [
    ("..\\new_dataset\\eclipse", "..\\new_dataset\\eclipse"),
    #("..\\new_dataset\\eclipse_test", "..\\new_dataset\\eclipse_test"),
    #("..\\new_dataset\\firefox", "..\\new_dataset\\firefox"),
    #("..\\new_dataset\\netbeans", "..\\new_dataset\\netbeans"),
    #("..\\new_dataset\\openoffice", "..\\new_dataset\\openoffice")
]

In [4]:
# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Concatenate preprocessed descriptions from all files in all directories
all_preprocessed_desc = []

# Iterate over each directory
for source_dir, target_dir in directories:
    # Iterate over each file in the directory
    for file_name in os.listdir(source_dir):
        print(file_name)

        # Check if the file is a CSV file        
        if file_name.endswith(".csv"):
            # Load the CSV file
            df = pd.read_csv(os.path.join(source_dir, file_name))
            
            # Check if the DataFrame is empty (end of file reached)
            if df.empty:
                print("End of file reached for:", file_name)
                continue

            # Get the data from the preprocessed_description column
            short_desc = df['preprocessed_description'].astype(str)

            # Concatenate the preprocessed descriptions
            all_preprocessed_desc.extend(short_desc)

    print("Preprocessing done for all files in:", source_dir)

# Fit TF-IDF vectorizer on combined data to build the vocabulary
tfidf_vectorizer.fit(all_preprocessed_desc)



eclipse_small_new.csv
eclipse_small_new_updated.csv
Preprocessing done for all files in: ..\new_dataset\eclipse


TfidfVectorizer()

In [5]:
# Iterate over each directory
for source_dir, target_dir in directories:
    # Iterate over each file in the directory
    for file_name in os.listdir(source_dir):
        print(file_name)

        # Check if the file is a CSV file        
        if file_name.endswith(".csv"):
            # Load the CSV file
            df = pd.read_csv(os.path.join(source_dir, file_name))
            
            # Check if the DataFrame is empty (end of file reached)
            if df.empty:
                print("End of file reached for:", file_name)
                continue

            # loop over all the rows in the dataframe except the row with issue_id 214301
            for index, row in df.iterrows():
                if row['bug_id'] == 214990:
                    description1 = row['preprocessed_description']
                    continue

                description2 = row['preprocessed_description']

                # convert the two descriptions into a string
                description1 = str(description1)
                description2 = str(description2)
                
                # Transform the two descriptions into TF-IDF vectors
                desc1_tfidf = tfidf_vectorizer.transform([description1])
                desc2_tfidf = tfidf_vectorizer.transform([description2])

                # Compute cosine similarity between the two descriptions
                similarity_score = cosine_similarity(desc1_tfidf, desc2_tfidf)

                # write the outpur in text file in the target directory
                #with open(os.path.join(target_dir, file_name), 'a') as f:
                #    f.write(str(row['bug_id']) + "," + str(214990) + "," + str(similarity_score[0][0]) + "\n")

                if row['bug_id'] == 215040:
                    print("Target ================================================================================")
                # print the id of the two descriptions and the cosine similarity between them
                print("Bug ID1:", row['bug_id'])
                print("Bug ID2:", 214990)
                print("Cosine Similarity between the two descriptions:", similarity_score[0][0])


    print("Preprocessing done for all files in:", source_dir)



eclipse_small_new.csv
Bug ID1: 214065
Bug ID2: 214990
Cosine Similarity between the two descriptions: 0.08753368230293479
Bug ID1: 214070
Bug ID2: 214990
Cosine Similarity between the two descriptions: 0.005242519057609486
Bug ID1: 214068
Bug ID2: 214990
Cosine Similarity between the two descriptions: 1.0
Bug ID1: 214072
Bug ID2: 214990
Cosine Similarity between the two descriptions: 0.4554540438108013
Bug ID1: 214071
Bug ID2: 214990
Cosine Similarity between the two descriptions: 0.02107139059882321
Bug ID1: 214069
Bug ID2: 214990
Cosine Similarity between the two descriptions: 1.0
Bug ID1: 214066
Bug ID2: 214990
Cosine Similarity between the two descriptions: 0.011257751532200235
Bug ID1: 214067
Bug ID2: 214990
Cosine Similarity between the two descriptions: 0.036245654225657814
Bug ID1: 214076
Bug ID2: 214990
Cosine Similarity between the two descriptions: 0.010146855710347649
Bug ID1: 214075
Bug ID2: 214990
Cosine Similarity between the two descriptions: 0.02144989116261487
Bug ID1

In [6]:
# Choose two preprocessed descriptions for similarity comparison
description1= """
build id m step to reproduce my environment are a follow session eclipse buildid m java version java vendor sun microsystems inc bootloader constant o linux , arch x , w gtk , nl en u command line argument o linux w gtk arch x user import to import a new project , any project is ok , then open another project , usually in about five minute then open the eclipse again , the problem will repeat more information the hole log file a follow session eclipse buildid m java version java vendor sun microsystems inc bootloader constant o linux , arch x , w gtk , nl en u command line argument o linux w gtk arch x entry org eclipse osgi message while loading class org eclipse mylyn internal task ui itaskhighlighter , thread thread worker , ,main timed out waiting m for thread thread worker , ,main to finish starting bundle update plugins org eclipse mylyn task ui v jar to avoid deadlock , thread thread worker , ,main is proceeding but org eclipse mylyn internal task ui itaskhighlighter may not be fully initialized stack org osgi framework bundleexception state change in progress for bundle update plugins org eclipse mylyn task ui v jar by thread worker at org eclipse osgi framework internal core abstractbundle beginstatechange abstractbundle java at org eclipse osgi framework internal core abstractbundle start abstractbundle java at org eclipse osgi framework util secureaction start secureaction java at org eclipse core runtime internal adaptor eclipselazystarter postfindlocalclass eclipselazystarter java at org eclipse osgi baseadaptor loader classpathmanager findlocalclass classpathmanager java at org eclipse osgi internal baseadaptor defaultclassloader findlocalclass defaultclassloader java at org eclipse osgi framework internal core bundleloader findlocalclass bundleloader java at org eclipse osgi framework internal core singlesourcepackage loadclass singlesourcepackage java at org eclipse osgi framework internal core bundleloader findclassinternal bundleloader java at org eclipse osgi framework internal core bundleloader findclass bundleloader java at org eclipse osgi framework internal core bundleloader findclass bundleloader java at org eclipse osgi internal baseadaptor defaultclassloader loadclass defaultclassloader java at java lang classloader loadclass classloader java at java lang classloader loadclassinternal classloader java at java lang class getdeclaredconstructors native method at java lang class privategetdeclaredconstructors class java at java lang class getconstructor class java at java lang class newinstance class java at java lang class newinstance class java at org eclipse osgi framework internal core abstractbundle loadbundleactivator abstractbundle java at org eclipse osgi framework internal core bundlecontextimpl start bundlecontextimpl java at org eclipse osgi framework internal core bundlehost startworker bundlehost java at org eclipse osgi framework internal core abstractbundle start abstractbundle java at org eclipse osgi framework util secureaction start secureaction java at org eclipse core runtime internal adaptor eclipselazystarter postfindlocalclass eclipselazystarter java at org eclipse osgi baseadaptor loader classpathmanager findlocalclass classpathmanager java at org eclipse osgi internal baseadaptor defaultclassloader findlocalclass defaultclassloader java at org eclipse osgi framework internal core bundleloader findlocalclass bundleloader java at org eclipse osgi framework internal core bundleloader findclassinternal bundleloader java at org eclipse osgi framework internal core bundleloader findclass bundleloader java at org eclipse osgi framework internal core bundleloader findclass bundleloader java at org eclipse osgi internal baseadaptor defaultclassloader loadclass defaultclassloader java at java lang classloader loadclass classloader java at org eclipse osgi framework internal core bundleloader loadclass bundleloader java at org eclipse osgi framework internal core bundlehost loadclass bundlehost java at org eclipse osgi framework internal core abstractbundle loadclass abstractbundle java at org eclipse core internal registry osgi registrystrategyosgi createexecutableextension registrystrategyosgi java at org eclipse core internal registry extensionregistry createexecutableextension extensionregistry java at org eclipse core internal registry configurationelement createexecutableextension configurationelement java at org eclipse core internal registry configurationelementhandle createexecutableextension configurationelementhandle java at org eclipse ui internal workbenchplugin run workbenchplugin java at org eclipse swt custom busyindicator showwhile busyindicator java at org eclipse ui internal workbenchplugin createextension workbenchplugin java at org eclipse ui internal decorator lightweightdecoratordefinition run lightweightdecoratordefinition java at org eclipse core runtime saferunner run saferunner java at org eclipse core runtime platform run platform java at org eclipse ui internal decorator lightweightdecoratordefinition internalgetdecorator lightweightdecoratordefinition java at org eclipse ui internal decorator lightweightdecoratordefinition decorate lightweightdecoratordefinition java at org eclipse ui internal decorator lightweightdecoratormanager lightweightrunnable run lightweightdecoratormanager java at org eclipse core runtime saferunner run saferunner java at org eclipse core runtime platform run platform java at org eclipse ui internal decorator lightweightdecoratormanager decorate lightweightdecoratormanager java at org eclipse ui internal decorator lightweightdecoratormanager getdecorations lightweightdecoratormanager java at org eclipse ui internal decorator decorationscheduler ensureresultcached decorationscheduler java at org eclipse ui internal decorator decorationscheduler run decorationscheduler java at org eclipse core internal job worker run worker java caused by org eclipse osgi framework internal core abstractbundle bundlestatusexception more root exception org eclipse osgi framework internal core abstractbundle bundlestatusexception at org eclipse osgi framework internal core abstractbundle beginstatechange abstractbundle java at org eclipse osgi framework internal core abstractbundle start abstractbundle java at org eclipse osgi framework util secureaction start secureaction java at org eclipse core runtime internal adaptor eclipselazystarter postfindlocalclass eclipselazystarter java at org eclipse osgi baseadaptor loader classpathmanager findlocalclass classpathmanager java at org eclipse osgi internal baseadaptor defaultclassloader findlocalclass defaultclassloader java at org eclipse osgi framework internal core bundleloader findlocalclass bundleloader java at org eclipse osgi framework internal core singlesourcepackage loadclass singlesourcepackage java at org eclipse osgi framework internal core bundleloader findclassinternal bundleloader java at org eclipse osgi framework internal core bundleloader findclass bundleloader java at org eclipse osgi framework internal core bundleloader findclass bundleloader java at org eclipse osgi internal baseadaptor defaultclassloader loadclass defaultclassloader java at java lang classloader loadclass classloader java at java lang classloader loadclassinternal classloader java at java lang class getdeclaredconstructors native method at java lang class privategetdeclaredconstructors class java at java lang class getconstructor class java at java lang class newinstance class java at java lang class newinstance class java at org eclipse osgi framework internal core abstractbundle loadbundleactivator abstractbundle java at org eclipse osgi framework internal core bundlecontextimpl start bundlecontextimpl java at org eclipse osgi framework internal core bundlehost startworker bundlehost java at org eclipse osgi framework internal core abstractbundle start abstractbundle java at org eclipse osgi framework util secureaction start secureaction java at org eclipse core runtime internal adaptor eclipselazystarter postfindlocalclass eclipselazystarter java at org eclipse osgi baseadaptor loader classpathmanager findlocalclass classpathmanager java at org eclipse osgi internal baseadaptor defaultclassloader findlocalclass defaultclassloader java at org eclipse osgi framework internal core bundleloader findlocalclass bundleloader java at org eclipse osgi framework internal core bundleloader findclassinternal bundleloader java at org eclipse osgi framework internal core bundleloader findclass bundleloader java at org eclipse osgi framework internal core bundleloader findclass bundleloader java at org eclipse osgi internal baseadaptor defaultclassloader loadclass defaultclassloader java at java lang classloader loadclass classloader java at org eclipse osgi framework internal core bundleloader loadclass bundleloader java at org eclipse osgi framework internal core bundlehost loadclass bundlehost java at org eclipse osgi framework internal core abstractbundle loadclass abstractbundle java at org eclipse core internal registry osgi registrystrategyosgi createexecutableextension registrystrategyosgi java at org eclipse core internal registry extensionregistry createexecutableextension extensionregistry java at org eclipse core internal registry configurationelement createexecutableextension configurationelement java at org eclipse core internal registry configurationelementhandle createexecutableextension configurationelementhandle java at org eclipse ui internal workbenchplugin run workbenchplugin java at org eclipse swt custom busyindicator showwhile busyindicator java at org eclipse ui internal workbenchplugin createextension workbenchplugin java at org eclipse ui internal decorator lightweightdecoratordefinition run lightweightdecoratordefinition java at org eclipse core runtime saferunner run saferunner java at org eclipse core runtime platform run platform java at org eclipse ui internal decorator lightweightdecoratordefinition internalgetdecorator lightweightdecoratordefinition java at org eclipse ui internal decorator lightweightdecoratordefinition decorate lightweightdecoratordefinition java at org eclipse ui internal decorator lightweightdecoratormanager lightweightrunnable run lightweightdecoratormanager java at org eclipse core runtime saferunner run saferunner java at org eclipse core runtime platform run platform java at org eclipse ui internal decorator lightweightdecoratormanager decorate lightweightdecoratormanager java at org eclipse ui internal decorator lightweightdecoratormanager getdecorations lightweightdecoratormanager java at org eclipse ui internal decorator decorationscheduler ensureresultcached decorationscheduler java at org eclipse ui internal decorator decorationscheduler run decorationscheduler java at org eclipse core internal job worker run worker java
"""
description2 = """
error log date fri jan cet message could not load tasklist hyperlink detector extension severity info plugin id org eclipse mylyn stack trace org eclipse core runtime coreexception plug in org eclipse mylyn java ui wa unable to load class org eclipse mylyn internal java ui javastacktracehyperlinkdetector at org eclipse core internal registry osgi registrystrategyosgi throwexception registrystrategyosgi java at org eclipse core internal registry osgi registrystrategyosgi createexecutableextension registrystrategyosgi java at org eclipse core internal registry extensionregistry createexecutableextension extensionregistry java at org eclipse core internal registry configurationelement createexecutableextension configurationelement java at org eclipse core internal registry configurationelementhandle createexecutableextension configurationelementhandle java at org eclipse mylyn internal task ui util tasksuiextensionreader readhyperlinkdetector tasksuiextensionreader java at org eclipse mylyn internal task ui util tasksuiextensionreader initstartupextensions tasksuiextensionreader java at org eclipse mylyn task ui tasksuiplugin start tasksuiplugin java at org eclipse osgi framework internal core bundlecontextimpl run bundlecontextimpl java at java security accesscontroller doprivileged native method at org eclipse osgi framework internal core bundlecontextimpl startactivator bundlecontextimpl java at org eclipse osgi framework internal core bundlecontextimpl start bundlecontextimpl java at org eclipse osgi framework internal core bundlehost startworker bundlehost java at org eclipse osgi framework internal core abstractbundle start abstractbundle java at org eclipse osgi framework util secureaction run secureaction java at java security accesscontroller doprivileged native method at org eclipse osgi framework util secureaction start secureaction java at org eclipse core runtime internal adaptor eclipselazystarter postfindlocalclass eclipselazystarter java at org eclipse osgi baseadaptor loader classpathmanager findlocalclass classpathmanager java at org eclipse osgi internal baseadaptor defaultclassloader findlocalclass defaultclassloader java at org eclipse osgi framework internal core bundleloader findlocalclass bundleloader java at org eclipse osgi framework internal core singlesourcepackage loadclass singlesourcepackage java at org eclipse osgi framework internal core bundleloader findclassinternal bundleloader java at org eclipse osgi framework internal core bundleloader findclass bundleloader java at org eclipse osgi framework internal core bundleloader findclass bundleloader java at org eclipse osgi internal baseadaptor defaultclassloader loadclass defaultclassloader java at java lang classloader loadclass classloader java at java lang classloader loadclassinternal classloader java at java lang class getdeclaredconstructors native method at java lang class privategetdeclaredconstructors class java at java lang class getconstructor class java at java lang class newinstance class java at java lang class newinstance class java at org eclipse osgi framework internal core abstractbundle loadbundleactivator abstractbundle java at org eclipse osgi framework internal core bundlecontextimpl start bundlecontextimpl java at org eclipse osgi framework internal core bundlehost startworker bundlehost java at org eclipse osgi framework internal core abstractbundle start abstractbundle java at org eclipse osgi framework util secureaction run secureaction java at java security accesscontroller doprivileged native method at org eclipse osgi framework util secureaction start secureaction java at org eclipse core runtime internal adaptor eclipselazystarter postfindlocalclass eclipselazystarter java at org eclipse osgi baseadaptor loader classpathmanager findlocalclass classpathmanager java at org eclipse osgi internal baseadaptor defaultclassloader findlocalclass defaultclassloader java at org eclipse osgi framework internal core bundleloader findlocalclass bundleloader java at org eclipse osgi framework internal core bundleloader findclassinternal bundleloader java at org eclipse osgi framework internal core bundleloader findclass bundleloader java at org eclipse osgi framework internal core bundleloader findclass bundleloader java at org eclipse osgi internal baseadaptor defaultclassloader loadclass defaultclassloader java at java lang classloader loadclass classloader java at org eclipse osgi framework internal core bundleloader loadclass bundleloader java at org eclipse osgi framework internal core bundlehost loadclass bundlehost java at org eclipse osgi framework internal core abstractbundle loadclass abstractbundle java at org eclipse core internal registry osgi registrystrategyosgi createexecutableextension registrystrategyosgi java at org eclipse core internal registry extensionregistry createexecutableextension extensionregistry java at org eclipse core internal registry configurationelement createexecutableextension configurationelement java at org eclipse core internal registry configurationelementhandle createexecutableextension configurationelementhandle java at org eclipse ui internal workbenchplugin run workbenchplugin java at org eclipse swt custom busyindicator showwhile busyindicator java at org eclipse ui internal workbenchplugin createextension workbenchplugin java at org eclipse ui internal decorator lightweightdecoratordefinition run lightweightdecoratordefinition java at org eclipse core runtime saferunner run saferunner java at org eclipse core runtime platform run platform java at org eclipse ui internal decorator lightweightdecoratordefinition internalgetdecorator lightweightdecoratordefinition java at org eclipse ui internal decorator lightweightdecoratordefinition decorate lightweightdecoratordefinition java at org eclipse ui internal decorator lightweightdecoratormanager lightweightrunnable run lightweightdecoratormanager java at org eclipse core runtime saferunner run saferunner java at org eclipse core runtime platform run platform java at org eclipse ui internal decorator lightweightdecoratormanager decorate lightweightdecoratormanager java at org eclipse ui internal decorator lightweightdecoratormanager getdecorations lightweightdecoratormanager java at org eclipse ui internal decorator decorationscheduler ensureresultcached decorationscheduler java at org eclipse ui internal decorator decorationscheduler run decorationscheduler java at org eclipse core internal job worker run worker java org eclipse core runtime coreexception org eclipse core runtime internal adaptor eclipselazystarter terminatingclassnotfoundexception an error occurred while automatically activating bundle org eclipse mylyn java ui at org eclipse core runtime internal adaptor eclipselazystarter postfindlocalclass eclipselazystarter java at org eclipse osgi baseadaptor loader classpathmanager findlocalclass classpathmanager java at org eclipse osgi internal baseadaptor defaultclassloader findlocalclass defaultclassloader java at org eclipse osgi framework internal core bundleloader findlocalclass bundleloader java at org eclipse osgi framework internal core bundleloader findclassinternal bundleloader java at org eclipse osgi framework internal core bundleloader findclass bundleloader java at org eclipse osgi framework internal core bundleloader findclass bundleloader java at org eclipse osgi internal baseadaptor defaultclassloader loadclass defaultclassloader java at java lang classloader loadclass classloader java at org eclipse osgi framework internal core bundleloader loadclass bundleloader java at org eclipse osgi framework internal core bundlehost loadclass bundlehost java at org eclipse osgi framework internal core abstractbundle loadclass abstractbundle java at org eclipse core internal registry osgi registrystrategyosgi createexecutableextension registrystrategyosgi java at org eclipse core internal registry extensionregistry createexecutableextension extensionregistry java at org eclipse core internal registry configurationelement createexecutableextension configurationelement java at org eclipse core internal registry configurationelementhandle createexecutableextension configurationelementhandle java at org eclipse mylyn internal task ui util tasksuiextensionreader readhyperlinkdetector tasksuiextensionreader java at org eclipse mylyn internal task ui util tasksuiextensionreader initstartupextensions tasksuiextensionreader java at org eclipse mylyn task ui tasksuiplugin start tasksuiplugin java at org eclipse osgi framework internal core bundlecontextimpl run bundlecontextimpl java at java security accesscontroller doprivileged native method at org eclipse osgi framework internal core bundlecontextimpl startactivator bundlecontextimpl java at org eclipse osgi framework internal core bundlecontextimpl start bundlecontextimpl java at org eclipse osgi framework internal core bundlehost startworker bundlehost java at org eclipse osgi framework internal core abstractbundle start abstractbundle java at org eclipse osgi framework util secureaction run secureaction java at java security accesscontroller doprivileged native method at org eclipse osgi framework util secureaction start secureaction java at org eclipse core runtime internal adaptor eclipselazystarter postfindlocalclass eclipselazystarter java at org eclipse osgi baseadaptor loader classpathmanager findlocalclass classpathmanager java at org eclipse osgi internal baseadaptor defaultclassloader findlocalclass defaultclassloader java at org eclipse osgi framework internal core bundleloader findlocalclass bundleloader java at org eclipse osgi framework internal core singlesourcepackage loadclass singlesourcepackage java at org eclipse osgi framework internal core bundleloader findclassinternal bundleloader java at org eclipse osgi framework internal core bundleloader findclass bundleloader java at org eclipse osgi framework internal core bundleloader findclass bundleloader java at org eclipse osgi internal baseadaptor defaultclassloader loadclass defaultclassloader java at java lang classloader loadclass classloader java at java lang classloader loadclassinternal classloader java at java lang class getdeclaredconstructors native method at java lang class privategetdeclaredconstructors class java at java lang class getconstructor class java at java lang class newinstance class java at java lang class newinstance class java at org eclipse osgi framework internal core abstractbundle loadbundleactivator abstractbundle java at org eclipse osgi framework internal core bundlecontextimpl start bundlecontextimpl java at org eclipse osgi framework internal core bundlehost startworker bundlehost java at org eclipse osgi framework internal core abstractbundle start abstractbundle java at org eclipse osgi framework util secureaction run secureaction java at java security accesscontroller doprivileged native method at org eclipse osgi framework util secureaction start secureaction java at org eclipse core runtime internal adaptor eclipselazystarter postfindlocalclass eclipselazystarter java at org eclipse osgi baseadaptor loader classpathmanager findlocalclass classpathmanager java at org eclipse osgi internal baseadaptor defaultclassloader findlocalclass defaultclassloader java at org eclipse osgi framework internal core bundleloader findlocalclass bundleloader java at org eclipse osgi framework internal core bundleloader findclassinternal bundleloader java at org eclipse osgi framework internal core bundleloader findclass bundleloader java at org eclipse osgi framework internal core bundleloader findclass bundleloader java at org eclipse osgi internal baseadaptor defaultclassloader loadclass defaultclassloader java at java lang classloader loadclass classloader java at org eclipse osgi framework internal core bundleloader loadclass bundleloader java at org eclipse osgi framework internal core bundlehost loadclass bundlehost java at org eclipse osgi framework internal core abstractbundle loadclass abstractbundle java at org eclipse core internal registry osgi registrystrategyosgi createexecutableextension registrystrategyosgi java at org eclipse core internal registry extensionregistry createexecutableextension extensionregistry java at org eclipse core internal registry configurationelement createexecutableextension configurationelement java at org eclipse core internal registry configurationelementhandle createexecutableextension configurationelementhandle java at org eclipse ui internal workbenchplugin run workbenchplugin java at org eclipse swt custom busyindicator showwhile busyindicator java at org eclipse ui internal workbenchplugin createextension workbenchplugin java at org eclipse ui internal decorator lightweightdecoratordefinition run lightweightdecoratordefinition java at org eclipse core runtime saferunner run saferunner java at org eclipse core runtime platform run platform java at org eclipse ui internal decorator lightweightdecoratordefinition internalgetdecorator lightweightdecoratordefinition java at org eclipse ui internal decorator lightweightdecoratordefinition decorate lightweightdecoratordefinition java at org eclipse ui internal decorator lightweightdecoratormanager lightweightrunnable run lightweightdecoratormanager java at org eclipse core runtime saferunner run saferunner java at org eclipse core runtime platform run platform java at org eclipse ui internal decorator lightweightdecoratormanager decorate lightweightdecoratormanager java at org eclipse ui internal decorator lightweightdecoratormanager getdecorations lightweightdecoratormanager java at org eclipse ui internal decorator decorationscheduler ensureresultcached decorationscheduler java at org eclipse ui internal decorator decorationscheduler run decorationscheduler java at org eclipse core internal job worker run worker java caused by org osgi framework bundleexception the activator org eclipse mylyn internal java ui javauibridgeplugin for bundle org eclipse mylyn java ui is invalid at org eclipse osgi framework internal core abstractbundle loadbundleactivator abstractbundle java at org eclipse osgi framework internal core bundlecontextimpl start bundlecontextimpl java at org eclipse osgi framework internal core bundlehost startworker bundlehost java at org eclipse osgi framework internal core abstractbundle start abstractbundle java at org eclipse osgi framework util secureaction run secureaction java at java security accesscontroller doprivileged native method at org eclipse osgi framework util secureaction start secureaction java at org eclipse core runtime internal adaptor eclipselazystarter postfindlocalclass eclipselazystarter java more caused by java lang noclassdeffounderror org eclipse mylyn internal context ui abstractcontextuiplugin at java lang classloader defineclass native method at java lang classloader defineclass classloader java at org eclipse osgi internal baseadaptor defaultclassloader defineclass defaultclassloader java at org eclipse osgi baseadaptor loader classpathmanager defineclass classpathmanager java at org eclipse osgi baseadaptor loader classpathmanager findclassimpl classpathmanager java at org eclipse osgi baseadaptor loader classpathmanager findlocalclassimpl classpathmanager java at org eclipse osgi baseadaptor loader classpathmanager findlocalclass classpathmanager java at org eclipse osgi internal baseadaptor defaultclassloader findlocalclass defaultclassloader java at org eclipse osgi framework internal core bundleloader findlocalclass bundleloader java at org eclipse osgi framework internal core bundleloader findclassinternal bundleloader java at org eclipse osgi framework internal core bundleloader findclass bundleloader java at org eclipse osgi framework internal core bundleloader findclass bundleloader java at org eclipse osgi internal baseadaptor defaultclassloader loadclass defaultclassloader java at java lang classloader loadclass classloader java at org eclipse osgi framework internal core bundleloader loadclass bundleloader java at org eclipse osgi framework internal core bundlehost loadclass bundlehost java at org eclipse osgi framework internal core abstractbundle loadbundleactivator abstractbundle java more
"""
# Transform the two descriptions into TF-IDF vectors
desc1_tfidf = tfidf_vectorizer.transform([description1])
desc2_tfidf = tfidf_vectorizer.transform([description2])

# Compute cosine similarity between the two descriptions
similarity_score = cosine_similarity(desc1_tfidf, desc2_tfidf)

print("Cosine Similarity between the two descriptions:", similarity_score[0][0])


Cosine Similarity between the two descriptions: 0.9829924089454122
